In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
Grok API Key not set (and this is optional)


In [3]:
# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)

In [18]:
OPENAI_MODEL = "gpt-5-mini"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
GROK_MODEL = "grok-4"
GEMINI_MODEL = "gemini-2.5-pro"

In [5]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Windows',
  'arch': 'AMD64',
  'release': '11',
  'version': '10.0.22631',
  'kernel': '11',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': ''},
 'package_managers': ['winget', 'choco'],
 'cpu': {'brand': 'AMD Ryzen 5 5600H with Radeon Graphics',
  'cores_logical': 12,
  'cores_physical': 6,
  'simd': []},
 'toolchain': {'compilers': {'gcc': '', 'g++': '', 'clang': '', 'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': ''},
  'linkers': {'ld_lld': ''}}}

In [ ]:
message = f"""
Here is a report of the system information for my computer.
I want to run a Pytorch compiler to compile a single Pytorch file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any Pytorch compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile Pytorch code, then I'd like to run something like this in Python to compile and execute the code:

Please tell me exactly what I should use for the compile_command and run_command.

"""

response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))
    

You don’t currently have a C++ compiler installed. You’ll need to install one.

Simplest setup on your Windows 11 system (recommended): Microsoft C++ Build Tools (MSVC) via winget

1) Open an elevated PowerShell (Run as administrator).
2) Install the C++ build tools (includes the Windows SDK so MSVC can build C++ out of the box):
   winget install -e --id Microsoft.VisualStudio.2022.BuildTools --override "--quiet --wait --norestart --nocache --add Microsoft.VisualStudio.Workload.VCTools --includeRecommended"
3) After installation, you can compile by first initializing the compiler environment using vcvars64.bat.

Exact Python commands to compile and run main.cpp at high optimization

- This uses MSVC with link-time optimization for strong runtime performance.

compile_command = [
    "cmd", "/d", "/s", "/c",
    r'"C:\Program Files\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\Build\vcvars64.bat" && cl /nologo /O2 /GL /EHsc /std:c++20 /MD main.cpp /link /LTCG /OUT:main.exe'
]
run_command = ["main.exe"]

Notes:
- vcvars64.bat sets the necessary environment variables for cl to work. The path shown is the default for VS 2022 Build Tools; if you installed to a custom location, adjust that path accordingly.
- /O2 and /GL (with /LTCG at link) enable high optimization levels for good runtime performance. If you prefer faster compilation over maximum runtime speed, you can drop /GL and /LTCG.

In [7]:
system_prompt = """
Your task is to convert Python code into high performance Pytorch
Respond only with Pytorch Do not provide any explanation other than occasional comments.
The Pytorch response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to Pytorch with the fastest possible implementation that produces identical output in the least time.

Respond only with Pytorch
Python code to port:

```python
{python}
```
"""

In [8]:
def messages_for(python):
  return  [
    {"role":"system","content":system_prompt},
    {"role":"user","content":user_prompt_for(python)}
  ]



In [25]:
def write_reply(reply):
  with open ("pytorch.py","w",encoding="utf-8") as f:
    f.write(reply)


In [26]:
def port(client,model,python):
  response = client.chat.completions.create(model=model,messages=messages_for(python))
  reply = response.choices[0].message.content
  write_reply(reply)

In [14]:
pi = """
import numpy as np
import time

# Data
x = np.random.rand(1_000_000)
w = np.random.rand(1_000_000)
b = 0.1

# Timing
start = time.perf_counter()

y = np.dot(x, w) + b

end = time.perf_counter()

print("NumPy Output:", y)
print("NumPy Time Taken:", end - start, "seconds")
"""

In [28]:
def run_python(code):
 
    exec(code)

In [29]:
run_python(pi)

NumPy Output: 249929.90530039204
NumPy Time Taken: 0.001779599999281345 seconds


In [30]:
port(openai, OPENAI_MODEL, pi)

In [34]:
port(anthropic, CLAUDE_MODEL, pi)
pt = """
import torch
import time

# Data size
n = 1_000_000

# Device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use double precision to match NumPy's default dtype
dtype = torch.float64

# Generate data on chosen device
x = torch.rand(n, dtype=dtype, device=device)
w = torch.rand(n, dtype=dtype, device=device)
b = 0.1

# Warm-up for CUDA (if used)
if device.type == "cuda":
    torch.cuda.synchronize()

# Timing
start = time.perf_counter()
y = torch.dot(x, w) + b
if device.type == "cuda":
    torch.cuda.synchronize()
end = time.perf_counter()

print("PyTorch Output:", y.item())
print("PyTorch Time Taken:", end - start, "seconds")
"""

In [41]:
run_python(pt)

PyTorch Output: 249771.3687180019
PyTorch Time Taken: 0.0034960999983013608 seconds


In [42]:
run_python(pi)

NumPy Output: 250193.68504644226
NumPy Time Taken: 0.0030337000025610905 seconds


In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [5]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)


False
None


In [6]:
import os
import sys
import gradio as gr
from dotenv import load_dotenv

c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
load_dotenv(override=True)
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"


In [8]:
from openai import OpenAI

openai = OpenAI()

anthropic = OpenAI(api_key=anthropic_api_key,base_url=anthropic_url)
openai = OpenAI(api_key=openai_key)
gemini = OpenAI(api_key = google_api_key,base_url=gemini_url)
ollama = OpenAI(base_url=ollama_url,api_key="None")


In [ ]:
models= ["gpt-5","gemini-2.5-pro","llama"]
clients = {"gpt-5":openai,"gemini-2.5-pro":gemini,"llama":ollama}

In [22]:
system_prompt = """
Your task is to convert Python code into high performance Pytorch
Respond only with Pytorch Do not provide any explanation other than occasional comments.
The Pytorch response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to Pytorch with the fastest possible implementation that produces identical output in the least time.

Respond only with Pytorch
Python code to port:

```python
{python}
```
"""

In [23]:
def messages_for(python):
  return [{"role":"user","content":user_prompt_for(python)},{"role":"system","content":system_prompt}]

In [24]:
def write_output(python):
  with open("output_code.py","w") as f:
    f.write(python)


In [25]:
def port(model,python):
  client = clients[model]
  res = client.chat.completions.create(model=model,messages = messages_for(python))
  reply = res.choices[0].message.content
  write_output(reply)
  return reply

In [26]:
with gr.Blocks() as ui:
  with gr.Row():
    python = gr.Textbox(label="Python code: ",lines=28,value=pi)
    pytorch = gr.Textbox(label="Pytorch code: ",lines=28)
  with gr.Row():
    model = gr.Dropdown(models,label="Select model",value=models[0])
    convert=gr.Button("Convert Code")
  
  convert.click(port,inputs=[model,python],outputs = [pytorch])
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\gradio\blocks.py", line 2106, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\gradio\blocks.py", line 1588, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\anyio\t

In [19]:
import torch
import time

# Select fastest device available
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float64  # match NumPy default precision

# Data
x = torch.rand(1_000_000, dtype=dtype, device=device)
w = torch.rand(1_000_000, dtype=dtype, device=device)
b = torch.tensor(0.1, dtype=dtype, device=device)

# Timing
if device == "cuda":
    torch.cuda.synchronize()
start = time.perf_counter()

y = torch.dot(x, w) + b

if device == "cuda":
    torch.cuda.synchronize()
end = time.perf_counter()

print("PyTorch Output:", y.item())
print("PyTorch Time Taken:", end - start, "seconds")

PyTorch Output: 249780.3131405094
PyTorch Time Taken: 0.030948400002671406 seconds


False